# Prepare San Juan data

In [1]:
import scipy as sp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels as stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
%matplotlib inline
import myutil_prep_iq as myutil
import warnings
warnings.filterwarnings('ignore')
import sys
print(sys.executable)

/Users/carlos2/anaconda/envs/dsdht/bin/python


In [2]:
import importlib
_ = importlib.reload(myutil)

## Prepare training data

In [3]:
dfx_train = myutil.get_indexed_dataset('data/dengue_features_train.csv')
dfy_train = myutil.get_indexed_dataset('data/dengue_labels_train.csv')
dftrain = myutil.set_index(pd.merge(dfx_train, dfy_train))
dftrain_iq, _ = myutil.split_dataset_by_city(dftrain)
dftrain_iq.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
yearweekofyear,,,,,,,,,,,,,,,,,,,,,
200026,iq,2000,26,2000-07-01,0.192886,0.132257,0.340886,0.247200,25.41,296.740000,...,92.418571,25.41,16.651429,8.928571,26.400000,10.775000,32.5,20.7,3.0,0
200027,iq,2000,27,2000-07-08,0.216833,0.276100,0.289457,0.241657,60.61,296.634286,...,93.581429,60.61,16.862857,10.314286,26.900000,11.566667,34.0,20.8,55.6,0
200028,iq,2000,28,2000-07-15,0.176757,0.173129,0.204114,0.128014,55.52,296.415714,...,95.848571,55.52,17.120000,7.385714,26.800000,11.466667,33.0,20.7,38.1,0
200029,iq,2000,29,2000-07-22,0.227729,0.145429,0.254200,0.200314,5.60,295.357143,...,87.234286,5.60,14.431429,9.114286,25.766667,10.533333,31.5,14.7,30.0,0
200030,iq,2000,30,2000-07-29,0.328643,0.322129,0.254371,0.361043,62.76,296.432857,...,88.161429,62.76,15.444286,9.500000,26.600000,11.480000,33.3,19.1,4.0,0


In [4]:
# remove nans
dftrain_iq = myutil.set_nan_to_week_mean(dftrain_iq.copy())
dftrain_iq.isnull().sum()

city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

###  Add total_cases as a feature

In [5]:
dftrain_iq = myutil.add_total_cases_feature(dftrain_iq)
dftrain_iq.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases2,total_cases
yearweekofyear,,,,,,,,,,,,,,,,,,,,,
200026,iq,2000,26,2000-07-01,0.192886,0.132257,0.340886,0.247200,25.41,296.740000,...,25.41,16.651429,8.928571,26.400000,10.775000,32.5,20.7,3.0,0,0
200027,iq,2000,27,2000-07-08,0.216833,0.276100,0.289457,0.241657,60.61,296.634286,...,60.61,16.862857,10.314286,26.900000,11.566667,34.0,20.8,55.6,0,0
200028,iq,2000,28,2000-07-15,0.176757,0.173129,0.204114,0.128014,55.52,296.415714,...,55.52,17.120000,7.385714,26.800000,11.466667,33.0,20.7,38.1,0,0
200029,iq,2000,29,2000-07-22,0.227729,0.145429,0.254200,0.200314,5.60,295.357143,...,5.60,14.431429,9.114286,25.766667,10.533333,31.5,14.7,30.0,0,0
200030,iq,2000,30,2000-07-29,0.328643,0.322129,0.254371,0.361043,62.76,296.432857,...,62.76,15.444286,9.500000,26.600000,11.480000,33.3,19.1,4.0,0,0


###  Remove location and timescale features (since timescale is index)

In [6]:
dftrain_iq.drop(['city','year','weekofyear','week_start_date'], axis=1, inplace=True)
dftrain_iq.head()

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,...,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases2,total_cases
yearweekofyear,,,,,,,,,,,,,,,,,,,,,
200026,0.192886,0.132257,0.340886,0.247200,25.41,296.740000,298.450000,295.184286,307.3,293.1,...,25.41,16.651429,8.928571,26.400000,10.775000,32.5,20.7,3.0,0,0
200027,0.216833,0.276100,0.289457,0.241657,60.61,296.634286,298.428571,295.358571,306.6,291.1,...,60.61,16.862857,10.314286,26.900000,11.566667,34.0,20.8,55.6,0,0
200028,0.176757,0.173129,0.204114,0.128014,55.52,296.415714,297.392857,295.622857,304.5,292.6,...,55.52,17.120000,7.385714,26.800000,11.466667,33.0,20.7,38.1,0,0
200029,0.227729,0.145429,0.254200,0.200314,5.60,295.357143,296.228571,292.797143,303.6,288.6,...,5.60,14.431429,9.114286,25.766667,10.533333,31.5,14.7,30.0,0,0
200030,0.328643,0.322129,0.254371,0.361043,62.76,296.432857,297.635714,293.957143,307.0,291.5,...,62.76,15.444286,9.500000,26.600000,11.480000,33.3,19.1,4.0,0,0


###  Remove features with 100% correlation

In [7]:
# Features “precipitation_amount_mm” and “reanalysis_sat_precip_amt_mm” were found to be 100% correlated (pearson), 
# so we drop the latter.
# Feature“reanalysis_dew_point_temp_k” and “reanalysis_specific_humidity_g_per_kg” are 99.85% correlated (pearson), 
# so we drop the latter.
dftrain_iq.drop(['reanalysis_sat_precip_amt_mm','reanalysis_specific_humidity_g_per_kg'], axis=1, inplace=True)
dftrain_iq.head()

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases2,total_cases
yearweekofyear,,,,,,,,,,,,,,,,,,,,
200026,0.192886,0.132257,0.340886,0.247200,25.41,296.740000,298.450000,295.184286,307.3,293.1,43.19,92.418571,8.928571,26.400000,10.775000,32.5,20.7,3.0,0,0
200027,0.216833,0.276100,0.289457,0.241657,60.61,296.634286,298.428571,295.358571,306.6,291.1,46.00,93.581429,10.314286,26.900000,11.566667,34.0,20.8,55.6,0,0
200028,0.176757,0.173129,0.204114,0.128014,55.52,296.415714,297.392857,295.622857,304.5,292.6,64.77,95.848571,7.385714,26.800000,11.466667,33.0,20.7,38.1,0,0
200029,0.227729,0.145429,0.254200,0.200314,5.60,295.357143,296.228571,292.797143,303.6,288.6,23.96,87.234286,9.114286,25.766667,10.533333,31.5,14.7,30.0,0,0
200030,0.328643,0.322129,0.254371,0.361043,62.76,296.432857,297.635714,293.957143,307.0,291.5,31.80,88.161429,9.500000,26.600000,11.480000,33.3,19.1,4.0,0,0


### Convert all temperature features to Celsius

In [8]:
dftrain_iq = myutil.convert_k_to_celsius(dftrain_iq.copy())
dftrain_iq.head()

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases2,total_cases
yearweekofyear,,,,,,,,,,,,,,,,,,,,
200026,0.192886,0.132257,0.340886,0.247200,25.41,23.590000,25.300000,22.034286,34.15,19.95,43.19,92.418571,8.928571,26.400000,10.775000,32.5,20.7,3.0,0,0
200027,0.216833,0.276100,0.289457,0.241657,60.61,23.484286,25.278571,22.208571,33.45,17.95,46.00,93.581429,10.314286,26.900000,11.566667,34.0,20.8,55.6,0,0
200028,0.176757,0.173129,0.204114,0.128014,55.52,23.265714,24.242857,22.472857,31.35,19.45,64.77,95.848571,7.385714,26.800000,11.466667,33.0,20.7,38.1,0,0
200029,0.227729,0.145429,0.254200,0.200314,5.60,22.207143,23.078571,19.647143,30.45,15.45,23.96,87.234286,9.114286,25.766667,10.533333,31.5,14.7,30.0,0,0
200030,0.328643,0.322129,0.254371,0.361043,62.76,23.282857,24.485714,20.807143,33.85,18.35,31.80,88.161429,9.500000,26.600000,11.480000,33.3,19.1,4.0,0,0


## Save training data

In [9]:
dftrain_iq.iloc[:,:-1].to_csv('data/dengue_features_train_with_outliers_iq.csv')
dftrain_iq.iloc[:,-1:].to_csv('data/dengue_labels_train_iq.csv')

======================================================================================================================

## Prepare test data

In [10]:
dfx_test = myutil.get_indexed_dataset('data/dengue_features_test.csv')
dfx_test = dfx_test[dfx_test['city']=='iq']
dfx_test.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
yearweekofyear,,,,,,,,,,,,,,,,,,,,,
201026,iq,2010,26,2010-07-02,0.183783,0.142500,0.225129,0.150214,82.29,297.648571,...,34.11,92.581429,82.29,17.654286,9.428571,27.440,10.760,33.8,21.5,11.2
201027,iq,2010,27,2010-07-09,0.291657,0.272267,0.330700,0.320914,25.30,298.224286,...,9.10,83.885714,25.30,16.320000,10.157143,27.025,9.625,33.0,21.2,8.9
201028,iq,2010,28,2010-07-16,0.208543,0.366457,0.212629,0.255514,62.14,297.955714,...,61.09,92.057143,62.14,18.030000,9.557143,26.950,10.350,33.4,21.6,22.6
201029,iq,2010,29,2010-07-23,0.089286,0.063214,0.122057,0.081957,47.80,295.715714,...,19.60,88.970000,47.80,15.394286,7.828571,26.900,9.700,33.3,14.2,4.8
201030,iq,2010,30,2010-07-30,0.306100,0.327683,0.250086,0.267914,56.30,298.502857,...,18.93,78.610000,56.30,15.468571,11.771429,27.050,11.850,33.5,16.9,3.0


In [11]:
# remove nans
dftest_iq = myutil.set_nan_to_week_mean(dfx_test.copy())
dftest_iq.isnull().sum()

city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

###  Remove location and timescale features (since timescale is index)

In [12]:
dftest_iq.drop(['city','year','weekofyear','week_start_date'], axis=1, inplace=True)
dftest_iq.head()

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
yearweekofyear,,,,,,,,,,,,,,,,,,,,
201026,0.183783,0.142500,0.225129,0.150214,82.29,297.648571,299.214286,296.118571,307.9,293.7,34.11,92.581429,82.29,17.654286,9.428571,27.440,10.760,33.8,21.5,11.2
201027,0.291657,0.272267,0.330700,0.320914,25.30,298.224286,299.464286,294.831429,307.7,292.2,9.10,83.885714,25.30,16.320000,10.157143,27.025,9.625,33.0,21.2,8.9
201028,0.208543,0.366457,0.212629,0.255514,62.14,297.955714,298.964286,296.417143,306.4,292.5,61.09,92.057143,62.14,18.030000,9.557143,26.950,10.350,33.4,21.6,22.6
201029,0.089286,0.063214,0.122057,0.081957,47.80,295.715714,296.642857,293.638571,306.4,289.3,19.60,88.970000,47.80,15.394286,7.828571,26.900,9.700,33.3,14.2,4.8
201030,0.306100,0.327683,0.250086,0.267914,56.30,298.502857,300.000000,293.892857,308.1,290.5,18.93,78.610000,56.30,15.468571,11.771429,27.050,11.850,33.5,16.9,3.0


###  Remove features with 100% correlation

In [13]:
# Features “precipitation_amount_mm” and “reanalysis_sat_precip_amt_mm” were found to be 100% correlated (pearson), 
# so we drop the latter.
# Feature“reanalysis_dew_point_temp_k” and “reanalysis_specific_humidity_g_per_kg” are 99.85% correlated (pearson), 
# so we drop the latter.
dftest_iq.drop(['reanalysis_sat_precip_amt_mm','reanalysis_specific_humidity_g_per_kg'], axis=1, inplace=True)
dftest_iq.head()

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
yearweekofyear,,,,,,,,,,,,,,,,,,
201026,0.183783,0.142500,0.225129,0.150214,82.29,297.648571,299.214286,296.118571,307.9,293.7,34.11,92.581429,9.428571,27.440,10.760,33.8,21.5,11.2
201027,0.291657,0.272267,0.330700,0.320914,25.30,298.224286,299.464286,294.831429,307.7,292.2,9.10,83.885714,10.157143,27.025,9.625,33.0,21.2,8.9
201028,0.208543,0.366457,0.212629,0.255514,62.14,297.955714,298.964286,296.417143,306.4,292.5,61.09,92.057143,9.557143,26.950,10.350,33.4,21.6,22.6
201029,0.089286,0.063214,0.122057,0.081957,47.80,295.715714,296.642857,293.638571,306.4,289.3,19.60,88.970000,7.828571,26.900,9.700,33.3,14.2,4.8
201030,0.306100,0.327683,0.250086,0.267914,56.30,298.502857,300.000000,293.892857,308.1,290.5,18.93,78.610000,11.771429,27.050,11.850,33.5,16.9,3.0


### Convert all temperature features to Celsius

In [14]:
dftest_iq = myutil.convert_k_to_celsius(dftest_iq.copy())
dftest_iq.head()

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
yearweekofyear,,,,,,,,,,,,,,,,,,
201026,0.183783,0.142500,0.225129,0.150214,82.29,24.498571,26.064286,22.968571,34.75,20.55,34.11,92.581429,9.428571,27.440,10.760,33.8,21.5,11.2
201027,0.291657,0.272267,0.330700,0.320914,25.30,25.074286,26.314286,21.681429,34.55,19.05,9.10,83.885714,10.157143,27.025,9.625,33.0,21.2,8.9
201028,0.208543,0.366457,0.212629,0.255514,62.14,24.805714,25.814286,23.267143,33.25,19.35,61.09,92.057143,9.557143,26.950,10.350,33.4,21.6,22.6
201029,0.089286,0.063214,0.122057,0.081957,47.80,22.565714,23.492857,20.488571,33.25,16.15,19.60,88.970000,7.828571,26.900,9.700,33.3,14.2,4.8
201030,0.306100,0.327683,0.250086,0.267914,56.30,25.352857,26.850000,20.742857,34.95,17.35,18.93,78.610000,11.771429,27.050,11.850,33.5,16.9,3.0


## Save test data

In [15]:
dftest_iq.to_csv('data/dengue_features_test_with_outliers_iq.csv')

In [16]:
dftest_iq.index.size

156